In [29]:
import json
import azurecredentials
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [30]:
credential = AzureKeyCredential(azurecredentials.key)
text_analytics_client = TextAnalyticsClient(endpoint=azurecredentials.endpoint, credential=credential)

In [31]:
# Code to call Text Analytics service to analyze sentiment in text
file_name = "./data/tweet3.json"

# Language Detection
with open(file_name, mode='rt') as file:
    content = json.load(file)

Tweet text

In [32]:
content['TweetText']

'RT @TgLa7: #Nato: stop Ungheria su ingresso della Svezia e Finlandia'

Language detection

In [37]:
detected_languages = text_analytics_client.detect_language([content['TweetText']])
iso_lang = detected_languages[0].primary_language.iso6391_name

'it'

Finding Key Phrases

In [35]:
text_analytics_client.extract_key_phrases([content['TweetText']], language=detected_language[0])

[DocumentError(id=0, error=TextAnalyticsError(code=UnsupportedLanguageCode, message=Invalid language code '{'id': '0', 'primary_language': DetectedLanguage(name=Italian, iso6391_name=it, confidence_score=0.93), 'warnings': [], 'statistics': None, 'is_error': False, 'kind': 'LanguageDetection'}'. Supported languages: af,am,ar,as,az,be,bg,bn,br,bs,ca,cs,cy,da,de,el,en,eo,es,et,eu,fa,fi,fil,fr,fy,ga,gd,gl,gu,ha,he,hi,hr,hu,hy,id,it,ja,jv,ka,kk,km,kn,ko,ku,ky,la,lo,lt,lv,mg,mk,ml,mn,mr,ms,my,ne,nl,no,om,or,pa,pl,ps,pt-BR,pt-PT,ro,ru,sa,sd,si,sk,sl,so,sq,sr,su,sv,sw,ta,te,th,tr,ug,uk,ur,uz,vi,xh,yi,zh-Hans,zh-Hant. For additional details see https://aka.ms/text-analytics/language-support?tabs=key-phrase-extraction, target=None), is_error=True)]

In [ ]:

Write-Host("***Detecting Language***")
$result = Invoke-RestMethod -Method Post `
          -Uri "$endpoint/text/analytics/v3.1/languages" `
          -Headers $headers `
          -Body $data | ConvertTo-Json -Depth 6

$analysis = ($result | ConvertFrom-Json)
$langName = $analysis.documents.detectedLanguage.name
$langCode = $analysis.documents.detectedLanguage.iso6391Name
$langScore = $analysis.documents.detectedLanguage.confidenceScore

Write-Host ("  - Language: $langName`n  - Code: $langCode`n  - Score: $langScore")

# Key Phrases

$data = @{
    'documents' = @(
        @{
            "id" = "1"
            "text" = "$txt"
        }
    )
} | ConvertTo-Json

write-host "`n`n***Finding Key Phrases***"
$result = Invoke-RestMethod -Method Post `
          -Uri "$endpoint/text/analytics/v3.1/keyPhrases" `
          -Headers $headers `
          -Body $data | ConvertTo-Json -Depth 6

$analysis = ($result | ConvertFrom-Json)

$keyPhrases = $analysis.documents.keyPhrases

Write-Host "  - Key Phrases: "
foreach($keyPhrase in $keyPhrases) {
    Write-Host ("    ", $keyPhrase)
}

# Sentiment

$data = @{
    'documents' = @(
        @{
            "id" = "1"
            "text" = "$txt"
        }
    )
} | ConvertTo-Json

write-host "`n`n***Analyzing Sentiment***"
$result = Invoke-RestMethod -Method Post `
          -Uri "$endpoint/text/analytics/v3.1/sentiment" `
          -Headers $headers `
          -Body $data | ConvertTo-Json -Depth 6

$analysis = ($result | ConvertFrom-Json)

$sentiment = $analysis.documents.sentiment
$positive = $analysis.documents.confidenceScores.positive
$neutral = $analysis.documents.confidenceScores.neutral
$negative = $analysis.documents.confidenceScores.negative

Write-Host ("  - A $sentiment sentiment based on these scores:`n    - Positive: $positive`n    - Neutral: $neutral`n    - Negative: $negative")

# Known Entities

$data = @{
    'documents' = @(
        @{
            "id" = "1"
            "text" = "$txt"
        }
    )
} | ConvertTo-Json

write-host "`n`n***Identifying known entities***"
$result = Invoke-RestMethod -Method Post `
          -Uri "$endpoint/text/analytics/v3.1/entities/linking" `
          -Headers $headers `
          -Body $data | ConvertTo-Json -Depth 6

$analysis = ($result | ConvertFrom-Json)

$entities = $analysis.documents[0].entities

foreach ($entity in $entities) {
    $entityName = $entity.name 
    $entityUrl = $entity.url 
    Write-Host ("  - $entityName : $entityUrl")
}